<a href="https://colab.research.google.com/github/JRicardoCardona/Poverty-Level-Classification-Costa-Rica/blob/main/notebooks/Data_Understanding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**ML-Classification-Costa-Rican-Poverty**

Collaborators:

* José Ricardo Cardona Quesada

* Francisco Monge Ortiz




---



---



# Etapa 1. Entendimiento de los Datos

El etendimiento de los datos es una etapa inicial que es crucial en el proceso de análisis de datos. Esta etapa contiene varios pasos que permiten una mayor comprensión de los datos con los que se van a trabajar. Los pasos que se van a realizar para garantizar un mejor análisis a partir del entendimiento de los datos son los siguientes: **recopilación inicial de los datos, descripción de los datos, exploración de los datos y el aseguramiento de la calidad de los datos**.

---

## 1.1 Recopilación Inicial de los datos

En esta sección se realizan distintos procedimientos para llevar a cabo la **recopilación inicial de los datos**. Lo primero que se realiza es la indentificación del dataset que se quiere utilizar para realizar el proyecto, en este caso se escogió uno sobre *predicciones de los niveles de pobreza en los hogares de Costa Rica*. Una vez se tienen el *dataset* con el que se va a trabajar, es necesario tener el acceso y carga a la fuente de datos (más adelante se indica la fuente de donde se obtuvo el *dataset* y cómo se descarga).

A continucación se importan las librerías necesarias y se descargan el *dataset* de entrenamiento.

In [ ]:
!pip install chart_studio
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import chart_studio.plotly as py
from plotly import tools
from plotly.subplots import make_subplots
from datetime import date
import pandas as pd
import numpy as np
import plotly.figure_factory as ff
import seaborn as sns
import matplotlib.pyplot as plt
import os


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 783.0 kB/s eta 0:00:00


Como se menciono anteriormente, el *dataset* a utilizar el sobre *los niveles de pobreza en los hogares de Costa Rica*. Este *dataset* se descargo de la página, en la sección de competencia, en Kaggle (https://www.kaggle.com/c/costa-rican-household-poverty-prediction/data) y se almacena localmente en la carpeta del proyecto.

Es **importante** contar con un *username* y el *kaggle_key* (llave de API) para poder descargar exitosamente los datasets.

**Dataset adquirido:**

*   train.csv: datos de entrenamiento con etiquetas de niveles de pobreza.



In [ ]:
# Se definen el usuario de kaggle y llave de API para descargar el dataset
os.environ['KAGGLE_USERNAME'] = 'josecardonaq03'
os.environ['KAGGLE_KEY'] = 'c1aee1d76bb0d0b108abea013cf75435'
!kaggle competitions download -c costa-rican-household-poverty-prediction
!unzip costa-rican-household-poverty-prediction.zip

  0% 0.00/2.33M [00:00<?, ?B/s]
100% 2.33M/2.33M [00:00<00:00, 41.2MB/s]
Archive:  costa-rican-household-poverty-prediction.zip
  inflating: codebook.csv            
  inflating: codebook.xlsx           
  inflating: sample_submission.csv   
  inflating: sample_submission.csv.zip  
  inflating: test.csv                
  inflating: test.csv.zip            
  inflating: train.csv               
  inflating: train.csv.zip           


Durante la carga de los datos no se encontraron problemas.

***Informe de Recopilación Inicial***

Este es un análisis inicial que salió de ver los datos en la página de Kaggle antes de cargarlos y para ver si realmente era un buen dataset

***¿Qué atributos (columnas) de los datos parecen más prometedores?***

Después de una revisión inicial del dataset, algunos de los atributos que parecen más prometedores para predecir el nivel de pobreza son:

**meaneduc**: años promedio de educación para adultos (18+). Un mayor nivel educativo suele correlacionarse con mejores oportunidades laborales e ingresos.

**overcrowding**: hacinamiento (# personas por habitación). Un alto nivel de hacinamiento es un claro indicador de pobreza.

**dependency**: tasa de dependencia (miembros <19 o >64 / miembros entre 19-64). Una alta tasa implica más dependientes por cada persona en edad laboral.

**edjefe**, **edjefa**: años de educación del jefe/a de hogar. Similar a meaneduc, el nivel educativo de quien lidera el hogar es clave.

**v2a1**: pago mensual de alquiler. Un monto muy bajo o nulo podría indicar precariedad de la vivienda.

**tipovivi1** a tipovivi5: tipo de vivienda (propia pagada, pagándose, alquilada, precaria, otra). La tenencia de una vivienda refleja el patrimonio del hogar.

***¿Qué atributos no parecen relevantes y se pueden excluir? ***

Algunas columnas que a priori no parecen aportar mucho valor predictivo son:

**Id, idhogar:** identificadores del registro y del hogar. No tienen significado por sí mismos.

**rez_esc:** años de retraso escolar. Ya se tiene la escolaridad como tal (escolari).

**age, SQBage:** edad de la persona y edad al cuadrado. No es claro que haya una relación directa o importante con el nivel de pobreza.

***¿Existen datos suficientes para obtener conclusiones generales o realizar predicciones precisas?***

El dataset de entrenamiento contiene 9557 registros, lo cual es una cantidad razonable para el tipo de problema. Habrá que analizar cuántos registros hay por cada categoría de la variable Target para asegurar que esté balanceado. De ser necesario, se podrían aplicar técnicas de sobremuestreo o submuestreo.

***¿Dispone de atributos suficientes para su método de modelado?***

El dataset cuenta con 142 atributos, abarcando diversos aspectos socioeconómicos y de las condiciones de vida de los hogares, por lo que en principio sí se dispondría de atributos suficientes para intentar predecir el nivel de pobreza. Se podría hacer un análisis de correlación con la variable objetivo y un proceso de selección de características para quedarse con los atributos más relevantes.

***¿Está fusionando varios orígenes de datos? En caso afirmativo, ¿existen áreas que puedan plantear problemas al fusionar?***

Para este proyecto, solo se usa el dataset provisto para la competencia, el cual ya viene integrado. No se están fusionando varias fuentes.
¿Ha considerado cómo se gestionan los valores perdidos en cada origen de datos?
Revisando las estadísticas del dataset, se observa que varias columnas (v2a1, v18q1, rez_esc, meaneduc, SQBmeaned) tienen una cantidad considerable de valores nulos. Será importante definir una estrategia para manejarlos, ya sea eliminando esos registros, imputando los valores faltantes con la media/mediana/moda u otro método según corresponda para cada variable, o creando una categoría adicional para representar el valor faltante.


---

## 1.2 Descripción de los Datos

En esta sección se busca describir los datos del *dataset* seleccionado de la manera más clara y concisa posible. Para esto primero se realiza una lectura del *dataset* descargado, para posteriormente poder ver la información general del conjunto de datos (como el número de filas y columnas, el uso de memoria y la verificación de las etiquetas en el *dataset*). Finalmente, apartir de esta información, se realiza un informe de descripción de los datos.

### Dataset entrenamiento

Se realiza la lectura del *.csv* que contiene el *dataset*.

In [ ]:
df_train = pd.read_csv('train.csv')
print("Primeras filas:\n")
df_train.head()


Primeras filas:



,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
0,ID_279628684,190000.0,0,3,0,1,1,0,NaN,0,...,100,1849,1,100,0,1.000000,0.0,100.0,1849,4
1,ID_f29eb3ddd,135000.0,0,4,0,1,1,1,1.0,0,...,144,4489,1,144,0,1.000000,64.0,144.0,4489,4
2,ID_68de51c94,NaN,0,8,0,1,1,0,NaN,0,...,121,8464,1,0,0,0.250000,64.0,121.0,8464,4
3,ID_d671db89c,180000.0,0,5,0,1,1,1,1.0,0,...,81,289,16,121,4,1.777778,1.0,121.0,289,4
4,ID_d56d6f5f5,180000.0,0,5,0,1,1,1,1.0,0,...,121,1369,16,121,4,1.777778,1.0,121.0,1369,4


Se procede a realizar la consultas de la información general del conjunto de datos. También se consulta el número de filas y columnas, así como el uso de memoria.

In [ ]:
# Obtener información sobre las columnas y tipos de datos
print("\nInformación de columnas y tipos de datos en train_data:\n")
print(df_train.info())
print()
# Obtener número de filas y columnas de df_train
print("Número de filas en df_train:", df_train.shape[0])
print()
print("Número de columnas en df_train:", df_train.shape[1])


# Obtener uso de memoria de df_train
print("\nUso de memoria de df_train:")
print(df_train.memory_usage())



Información de columnas y tipos de datos en train_data:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9557 entries, 0 to 9556
Columns: 143 entries, Id to Target
dtypes: float64(8), int64(130), object(5)
memory usage: 10.4+ MB
None

Número de filas en df_train: 9557

Número de columnas en df_train: 143

Uso de memoria de df_train:
Index                128
Id                 76456
v2a1               76456
hacdor             76456
rooms              76456
                   ...  
SQBovercrowding    76456
SQBdependency      76456
SQBmeaned          76456
agesq              76456
Target             76456
Length: 144, dtype: int64


Ahora, se procede a realizar una verificación de la columna *Target* en el *dataset*. Se puede observar si sí está presente y en cuántos registros se encuentra.

In [ ]:
# Se verifica que tenga la columna target (Etiqueta de los niveles de pobreza)
if 'Target' in df_train.columns:
    print("La columna 'Target' está presente en df_train.")
else:
    print("La columna 'Target' no está presente en df_train.")
print()
df_train['Target']

La columna 'Target' está presente en df_train.



0       4
1       4
2       4
3       4
4       4
       ..
9552    2
9553    2
9554    2
9555    2
9556    2
Name: Target, Length: 9557, dtype: int64

---

###Informe de Descripción de Datos

Al ver los primeros datos, claramente se pueden ver ciertos aspectos de mejora y que deberían ser analizados a mayor detalle:

1. Los nombres de las variables en el dataset, como v2a1, v18q1, SQBescolari, rez_esc, etc., no son muy informativos por sí mismos. Para facilitar la comprensión y el análisis, sería beneficioso cambiarlos por nombres más significativos que reflejen claramente qué representa cada variable. Por ejemplo, v2a1 podría renombrarse como "monthly_rent" si representa el alquiler mensual.

2. Se observa que varias columnas del dataset contienen valores nulos (NaN). Antes de proceder con el entrenamiento del modelo, es crucial decidir cómo manejar estos valores faltantes. Además de averiguar todas las columnas que contienen este tipo de datos.

3. El dataset contiene diversas columnas con diferentes tipos de datos, cada una de las cuales requiere un tratamiento específico durante el proceso de análisis y modelado. Por lo tanto, es fundamental realizar un estudio detallado de los tipos de datos presentes.

4. En este dataset también se puede identificar que hay ciertas columnas como "tipovivi1" que son de one hot encoding, lo que resulta muy beneficioso para eliminar la ambigüedad, evitar la asignación de orden y la reducción de sesgo.

5. Para comprender mejor los datos y sus relaciones, sería valioso realizar una exploración más profunda. Esto podría incluir visualizar las distribuciones de las variables, buscar correlaciones, identificar valores atípicos e investigar patrones interesantes.

**Cantidad de datos:**

El dataset de entrenamiento (train.csv) contiene 9557 filas (registros) y 143 columnas (atributos). Esto representa una cantidad considerable de datos para el modelado, permitiendo capturar una amplia variedad de escenarios y patrones. Sin embargo, también implica un mayor tiempo de procesamiento. Se podría considerar utilizar un subconjunto de los datos si fuera necesario mejorar la eficiencia.

** Tipos de valores:**

Al analizar los tipos de datos de cada columna en el dataframe, se encuentra que hay 3 tipos principales:

* **int64:** 130 columnas contienen valores enteros. Incluye variables como hacdor, rooms, r4h1, escolari, etc.

**float64:** 8 columnas tienen valores de punto flotante. Por ejemplo, v2a1, dependency, edjefe, meaneduc.

**object:** 5 columnas son de tipo object, que en pandas suele representar strings. Aquí se encuentran Id, idhogar y algunas variables categóricas.

Es importante prestar atención a estos tipos de datos, ya que cada uno requiere un tratamiento especial durante el preprocesamiento y modelado. Las variables numéricas podrían necesitar normalización, mientras que las categóricas deben ser codificadas apropiadamente.

**Esquemas de codificación:**

En el dataset se observan varios esquemas de codificación para representar características categóricas. Por ejemplo:

* Variables como paredblolad, pisomoscer, etc. usan 1 para indicar presencia y 0 para ausencia.

* lugar1 a lugar6 representan las diferentes regiones con 1 y 0.

* Variables como tipovivi1 a tipovivi5 son esquemas de one-hot-encoding para el tipo de vivienda.

* Otras como parentesco1 a parentesco12 también siguen un esquema one-hot para la relación con el jefe de hogar.

En general estos esquemas parecen consistentes dentro del dataset. Sin embargo, hay algunas variables como edjefe/edjefa y dependency que usan 'yes', 'no' y valores numéricos mezclados, lo cual requeriría un tratamiento especial.



---



### Cambio nombres de columnas

Como se menciono en la sección anterior, algunas columnas se podrían beneficiar de un nombre que permita un mejor entendimiento de los valores que abarcan. A continuación se listan las variables cuyos nombres se van a cambiar y luego el código donde se hace este cambio:

In [ ]:
df_train = df_train.rename(columns={
    'v2a1': 'monthly_rent',
    'hacdor': 'overcrowding_bedrooms',
    'rooms': 'total_rooms',
    'hacapo': 'overcrowding_rooms',
    'v14a': 'bathroom',
    'refrig': 'refrigerator',
    'v18q': 'owns_tablet',
    'v18q1': 'num_tablets',
    'r4h1': 'males_under_12',
    'r4h2': 'males_over_12',
    'r4h3': 'total_males',
    'r4m1': 'females_under_12',
    'r4m2': 'females_over_12',
    'r4m3': 'total_females',
    'r4t1': 'persons_under_12',
    'r4t2': 'persons_over_12',
    'r4t3': 'total_persons',
    'tipovivi1': 'own_fully_paid',
    'tipovivi2': 'own_paying_installments',
    'tipovivi3': 'rented',
    'tipovivi4': 'precarious',
    'tipovivi5': 'other_housing',
    'qmobilephone': 'num_mobile_phones',
    'lugar1': 'region_central',
    'lugar2': 'region_chorotega',
    'lugar3': 'region_pacifico_central',
    'lugar4': 'region_brunca',
    'lugar5': 'region_huetar_atlantica',
    'lugar6': 'region_huetar_norte',
    'area1': 'urban_zone',
    'area2': 'rural_zone'
})

In [ ]:
df_train.head()


,Id,monthly_rent,overcrowding_bedrooms,total_rooms,overcrowding_rooms,bathroom,refrigerator,owns_tablet,num_tablets,males_under_12,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
0,ID_279628684,190000.0,0,3,0,1,1,0,NaN,0,...,100,1849,1,100,0,1.000000,0.0,100.0,1849,4
1,ID_f29eb3ddd,135000.0,0,4,0,1,1,1,1.0,0,...,144,4489,1,144,0,1.000000,64.0,144.0,4489,4
2,ID_68de51c94,NaN,0,8,0,1,1,0,NaN,0,...,121,8464,1,0,0,0.250000,64.0,121.0,8464,4
3,ID_d671db89c,180000.0,0,5,0,1,1,1,1.0,0,...,81,289,16,121,4,1.777778,1.0,121.0,289,4
4,ID_d56d6f5f5,180000.0,0,5,0,1,1,1,1.0,0,...,121,1369,16,121,4,1.777778,1.0,121.0,1369,4


---

## 1.3 Exploración de los datos

La exploración de los datos es crucial para las etapas iniciales del analisis de datos, ya que es la etapa en la que se utilizan distintas técnicas de visualización que permiten identificar las diversas caracteristicas del dataset, así como la calidad y precisión de este mismo, para un mejor entendimiento de los datos.

En Machine Learning, es fundamental el conjunto de datos sobre el que se realizan los modelados. Esto es porque los modelos son entrenados con dichos datos y la precisión de los modelos se pueden ver afectados en caso de que los datos no hayan sido explorados anteriormente.

¿Por qué tiene tanto impacto en la precisión de un modelo? Bueno, esto ocurre porque la exploración de los datos permite identificar las variables que se encuentran en el dataset así como el rol que tienen dentro del dataset. Y es que a partir de distintos análisis que se pueden realizar por medio de distintos *plots, histograms, bar charts* o cualquier otro gráfico, se puede llegar a determinar tanto la frecuencia de las variables como la interacción que puede existir entre distintas variables.

Por otro lado, la exploración de los datos permiten detectar a tiempo aquellas variables que cuentan con una cantidad considerable de valores faltantes, lo que puede afectar la precisión del modelo si no se aborda de manera apropiada.

### Estadísticas del dataset

In [ ]:
df_train.describe()

,monthly_rent,overcrowding_bedrooms,total_rooms,overcrowding_rooms,bathroom,refrigerator,owns_tablet,num_tablets,males_under_12,males_over_12,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
count,2.697000e+03,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,2215.000000,9557.000000,9557.000000,...,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9552.000000,9557.000000,9557.000000
mean,1.652316e+05,0.038087,4.955530,0.023648,0.994768,0.957623,0.231767,1.404063,0.385895,1.559171,...,74.222769,1643.774302,19.132887,53.500262,3.844826,3.249485,3.900409,102.588867,1643.774302,3.302292
std,1.504571e+05,0.191417,1.468381,0.151957,0.072145,0.201459,0.421983,0.763131,0.680779,1.036574,...,76.777549,1741.197050,18.751395,78.445804,6.946296,4.129547,12.511831,93.516890,1741.197050,1.009565
min,0.000000e+00,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,1.000000
25%,8.000000e+04,0.000000,4.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,...,16.000000,289.000000,9.000000,0.000000,0.000000,1.000000,0.111111,36.000000,289.000000,3.000000
50%,1.300000e+05,0.000000,5.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,...,36.000000,961.000000,16.000000,36.000000,1.000000,2.250000,0.444444,81.000000,961.000000,4.000000
75%,2.000000e+05,0.000000,6.000000,0.000000,1.000000,1.000000,0.000000,2.000000,1.000000,2.000000,...,121.000000,2601.000000,25.000000,81.000000,4.000000,4.000000,1.777778,134.560010,2601.000000,4.000000
max,2.353477e+06,1.000000,11.000000,1.000000,1.000000,1.000000,1.000000,6.000000,5.000000,8.000000,...,441.000000,9409.000000,169.000000,441.000000,81.000000,36.000000,64.000000,1369.000000,9409.000000,4.000000


### Verificación de columnas binarias

Hay varias columnas que deben de tener valores binarios en el dataset, se revisa que todas cumplan esta restricción

In [ ]:

# List of columns that should have only 1 or 0 values
binary_cols = ['paredblolad', 'paredzocalo', 'paredpreb',
               'pareddes', 'paredmad', 'paredzinc', 'paredfibras', 'paredother', 'pisomoscer', 'pisocemento',
               'pisoother', 'pisonatur', 'pisonotiene', 'pisomadera', 'techozinc', 'techoentrepiso', 'techocane',
               'techootro', 'cielorazo', 'abastaguadentro', 'abastaguafuera', 'abastaguano', 'public', 'planpri',
               'noelec', 'coopele', 'sanitario1', 'sanitario2', 'sanitario3', 'sanitario5', 'sanitario6',
               'energcocinar1', 'energcocinar2', 'energcocinar3', 'energcocinar4', 'elimbasu1', 'elimbasu2',
               'elimbasu3', 'elimbasu4', 'elimbasu5', 'elimbasu6', 'epared1', 'epared2', 'epared3', 'etecho1',
               'etecho2', 'etecho3', 'eviv1', 'eviv2', 'eviv3', 'dis', 'male', 'female', 'estadocivil1',
               'estadocivil2', 'estadocivil3', 'estadocivil4', 'estadocivil5', 'estadocivil6', 'estadocivil7',
               'parentesco1', 'parentesco2', 'parentesco3', 'parentesco4', 'parentesco5', 'parentesco6',
               'parentesco7', 'parentesco8', 'parentesco9', 'parentesco10', 'parentesco11', 'parentesco12',
               'instlevel1', 'instlevel2', 'instlevel3', 'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7',
               'instlevel8', 'instlevel9', 'computer', 'television', 'mobilephone','overcrowding_bedrooms', 'overcrowding_rooms', 'bathroom', 'refrigerator', 'owns_tablet',
               'own_fully_paid', 'own_paying_installments', 'rented', 'precarious', 'other_housing',
               'region_central', 'region_chorotega', 'region_pacifico_central', 'region_brunca',
               'region_huetar_atlantica', 'region_huetar_norte', 'urban_zone', 'rural_zone' ]

contador_inv = 0
for col in binary_cols:
    invalid_values = df_train[col][(df_train[col] != 1) & (df_train[col] != 0)]
    if len(invalid_values) > 0:
        print(f"Column '{col}' has invalid values: {invalid_values.unique()}")
        contador_inv+=1

print(f"Cantidad de columnas binarias con valores inválidos: {contador_inv}")


Cantidad de columnas binarias con valores inválidos: 0


### Distribución de la variable Target

En esta sección se realiza una una exploración sobre el comportamiento que se puede observar de la variable *target*, para así poder asegurar que tiene resultados variados y que permiten un entrenamiento correcto para los modelos. En el siguiente gráfico, lo que se busca es poder visualizar la cantidad de registros con los que cuentan cada tipo de *target* en el dataset de entrenamiento.

In [ ]:
# Visualization of the 'Target' distribution in df_train
target_counts = df_train['Target'].value_counts()
print(target_counts)
names = ["NonVulnerable", "Vulnerable", "Moderate Poverty", "Extereme Poverty"]
trace = go.Bar(x=names, y=target_counts.values,marker=dict(color='blue', opacity = 0.6))
layout = go.Layout(title='Distribution of Target in training dataset',
                   xaxis=dict(title='Target'),
                   yaxis=dict(title='Count'), margin=dict(l=100), width=1000, height=600)
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

Target
4    5996
2    1597
3    1209
1     755
Name: count, dtype: int64


En el gráfico anterior, se puede observar la distribución de la variable *target* en el dataset de entrenamiento. Importante recordar lo siguiente sobre la variable *target*:

* 1 = pobreza extrema
* 2 = pobreza moderada
* 3 = hogar vulnerable
* 4 = hogar NO vulnerable

Dicho esto, el gráfico permite identificar que la mayor parte de los registros del dataset se encuentran con un *target* de 4, esto con una cifra bastante considerable, ya que de los 9557 registros del dataset, 5596 son de *target* 4. Por otro lado, se puede observar que los *target* 3 y 2 tienen cifras bastantes similares, con 1597 y 1209 registros respectivamente. Mientras que el *target* 1, como era de esperar, es que cuenta con la menor cantidad de registros, con solo 755.

El hecho de que la categoría "target=4" (hogares no vulnerables) representa aproximadamente el 58% de los registros del conjunto de datos, no es algo bueno para el modelaje, ya que podría sesgar los modelos predictivos hacia esa clase mayoritaria, haciendo que sean menos efectivos para identificar correctamente los casos de pobreza extrema, pobreza moderada y hogares vulnerables. Por esta razón más adelante en la sección de preparación de datos se abordará como se piensa manejar esto para evitar un sesgo

---

---


---


---

### Overview of the Relationship between Target and some variables

En esta sección se van a explorar distintas relaciones entre datos y el Target que pueden ser beneficiosos para el estudio de estos y así tener un mejor entendimiento de los mismos. También se espera que a partir de las relaciones que se van a explorar, se pueda lograr visualizar posibles variables que sean necesarias de tratar en la sección de preparación de los datos.

In [ ]:
# Define poverty levels for easier interpretation
poverty_levels = {1: 'Extreme Poverty', 2: 'Moderate Poverty', 3: 'Vulnerable', 4: 'Non-Vulnerable'}
df_train['Poverty Level'] = df_train['Target'].map(poverty_levels)

# Function to replace 0 with 'False' and 1 with 'True'
def replace_binary(x):
    return 'True' if x == 1 else 'False'




---



#### (Fran) Wall Materials vs Poverty Levels

In [ ]:
# 1. Wall Materials vs Poverty Levels
wall_materials = ['paredblolad', 'paredzocalo', 'paredpreb', 'pareddes', 'paredmad', 'paredzinc', 'paredfibras', 'paredother']

fig = make_subplots(rows=2, cols=2, subplot_titles=wall_materials[:4])
row, col = 1, 1

for material in wall_materials[:4]:
    df_train[f'{material}_label'] = df_train[material].apply(replace_binary)
    df_material = df_train.groupby(['Poverty Level', f'{material}_label']).size().unstack()
    df_material_perc = df_material.div(df_material.sum(axis=1), axis=0) * 100

    for i in df_material_perc.columns:
        fig.add_trace(
            go.Bar(
                x=df_material_perc.index,
                y=df_material_perc[i],
                name=f'{material}_{i}',
                text=df_material_perc[i].round(1),
                textposition='auto',
                hoverinfo='text',
                hovertext=[f'{val:.1f}%' for val in df_material_perc[i]]
            ),
            row=row, col=col
        )

    fig.update_xaxes(title_text='Poverty Level', row=row, col=col)
    fig.update_yaxes(title_text='Percentage', row=row, col=col)

    col += 1
    if col > 2:
        col = 1
        row += 1

fig.update_layout(height=800, width=1000, title_text="Wall Materials Distribution Across Poverty Levels")
fig.show()



---



####  Floor Materials vs Poverty Levels

In [ ]:
# 2. Flooring Materials vs Poverty Levels
floor_materials = ['pisomoscer', 'pisocemento', 'pisoother', 'pisonatur', 'pisonotiene', 'pisomadera']

fig = make_subplots(rows=2, cols=3, subplot_titles=floor_materials)
row, col = 1, 1

for material in floor_materials:
    df_train[f'{material}_label'] = df_train[material].apply(replace_binary)
    df_material = df_train.groupby(['Poverty Level', f'{material}_label']).size().unstack()
    df_material_perc = df_material.div(df_material.sum(axis=1), axis=0) * 100

    for i in df_material_perc.columns:
        fig.add_trace(
            go.Bar(
                x=df_material_perc.index,
                y=df_material_perc[i],
                name=f'{material}_{i}',
                text=df_material_perc[i].round(1),
                textposition='auto',
                hoverinfo='text',
                hovertext=[f'{val:.1f}%' for val in df_material_perc[i]]
            ),
            row=row, col=col
        )

    fig.update_xaxes(title_text='Poverty Level', row=row, col=col)
    fig.update_yaxes(title_text='Percentage', row=row, col=col)

    col += 1
    if col > 3:
        col = 1
        row += 1

fig.update_layout(height=800, width=1200, title_text="Flooring Materials Distribution Across Poverty Levels")
fig.show()



---



#### Roof Materials Distribution Across Poverty Levels

In [ ]:
# 3. Roof Materials vs Poverty Levels
roof_materials = ['techozinc', 'techoentrepiso', 'techocane', 'techootro']

fig = make_subplots(rows=2, cols=2, subplot_titles=roof_materials)
row, col = 1, 1

for material in roof_materials:
    df_train[f'{material}_label'] = df_train[material].apply(replace_binary)
    df_material = df_train.groupby(['Poverty Level', f'{material}_label']).size().unstack()
    df_material_perc = df_material.div(df_material.sum(axis=1), axis=0) * 100

    for i in df_material_perc.columns:
        fig.add_trace(
            go.Bar(
                x=df_material_perc.index,
                y=df_material_perc[i],
                name=f'{material}_{i}',
                text=df_material_perc[i].round(1),
                textposition='auto',
                hoverinfo='text',
                hovertext=[f'{val:.1f}%' for val in df_material_perc[i]]
            ),
            row=row, col=col
        )

    fig.update_xaxes(title_text='Poverty Level', row=row, col=col)
    fig.update_yaxes(title_text='Percentage', row=row, col=col)

    col += 1
    if col > 2:
        col = 1
        row += 1

fig.update_layout(height=800, width=1000, title_text="Roof Materials Distribution Across Poverty Levels")
fig.show()

Here it can be interpreted that most households, no matter their income level and povery level have a zinc roof. There are a few outliers, however its not enough to tell nothing about the data



---



#### Average Education Distribution Across Poverty Levels

ESTE MÁS ABAJO ESTÁ EL QUE NOSOTROS TENIAMOS, PERO NO COMO BOXPLOT. REVISAR SI EL YAPPING TODAVIA NOS SIRVE O SINO CAMBIARLO

In [ ]:
# 4. Average Education vs Poverty Levels
fig = go.Figure()

for poverty_level in df_train['Poverty Level'].unique():
    data = df_train[df_train['Poverty Level'] == poverty_level]['meaneduc']
    fig.add_trace(go.Box(y=data, name=poverty_level, boxpoints='all', jitter=0.3, pointpos=-1.8))

fig.update_layout(
    title='Distribution of Average Education Across Poverty Levels',
    yaxis_title='Average Education (Years)',
    xaxis_title='Poverty Level',
    height=600,
    width=800
)

fig.show()



---



#### Household Size Distribution Across Poverty Levels

In [ ]:
fig = go.Figure()

for poverty_level in df_train['Poverty Level'].unique():
    data = df_train[df_train['Poverty Level'] == poverty_level]['hogar_total']
    fig.add_trace(go.Box(y=data, name=poverty_level, boxpoints='all', jitter=0.3, pointpos=-1.8))

fig.update_layout(
    title='Distribution of Household Size Across Poverty Levels',
    yaxis_title='Household Size',
    xaxis_title='Poverty Level',
    height=600,
    width=800
)

fig.show()



---



#### Types of Electricty Across Poverty Levels

In [ ]:
electricity_types = ['public', 'planpri', 'noelec', 'coopele']
electricity_names = ['Public', 'Private Plant', 'No Electricity', 'Cooperative']

fig = make_subplots(rows=2, cols=2, subplot_titles=electricity_names)
row, col = 1, 1

for elec_type, name in zip(electricity_types, electricity_names):
    df_train[f'{elec_type}_label'] = df_train[elec_type].apply(replace_binary)
    df_elec = df_train.groupby(['Poverty Level', f'{elec_type}_label']).size().unstack()
    df_elec_perc = df_elec.div(df_elec.sum(axis=1), axis=0) * 100

    for i in df_elec_perc.columns:
        fig.add_trace(
            go.Bar(
                x=df_elec_perc.index,
                y=df_elec_perc[i],
                name=f'{name}_{i}',
                text=df_elec_perc[i].round(1),
                textposition='auto',
                hoverinfo='text',
                hovertext=[f'{val:.1f}%' for val in df_elec_perc[i]]
            ),
            row=row, col=col
        )

    fig.update_xaxes(title_text='Poverty Level', row=row, col=col)
    fig.update_yaxes(title_text='Percentage', row=row, col=col)

    col += 1
    if col > 2:
        col = 1
        row += 1

fig.update_layout(height=800, width=1000, title_text="Types of Electricity Across Poverty Levels")
fig.show()




---



#### (Fran) Types of Waste Disposal Across Poverty Levels

In [ ]:
waste_types = ['elimbasu1', 'elimbasu2', 'elimbasu3', 'elimbasu4', 'elimbasu5', 'elimbasu6']
waste_names = ['Garbage Truck', 'Buried', 'Burning', 'Throwing in Unoccupied Space', 'Throwing in River/Creek/Sea', 'Other']

fig = make_subplots(rows=2, cols=3, subplot_titles=waste_names)
row, col = 1, 1

for waste_type, name in zip(waste_types, waste_names):
    df_train[f'{waste_type}_label'] = df_train[waste_type].apply(replace_binary)
    df_waste = df_train.groupby(['Poverty Level', f'{waste_type}_label']).size().unstack()
    df_waste_perc = df_waste.div(df_waste.sum(axis=1), axis=0) * 100

    for i in df_waste_perc.columns:
        fig.add_trace(
            go.Bar(
                x=df_waste_perc.index,
                y=df_waste_perc[i],
                name=f'{name}_{i}',
                text=df_waste_perc[i].round(1),
                textposition='auto',
                hoverinfo='text',
                hovertext=[f'{val:.1f}%' for val in df_waste_perc[i]]
            ),
            row=row, col=col
        )

    fig.update_xaxes(title_text='Poverty Level', row=row, col=col)
    fig.update_yaxes(title_text='Percentage', row=row, col=col)

    col += 1
    if col > 3:
        col = 1
        row += 1

fig.update_layout(height=800, width=1200, title_text="Types of Waste Disposal Across Poverty Levels")
fig.show()


AQUI QUEMAR BASURA ES MAS COMUN PARA POBREZA Y USAR CAMION DE BASURA MENOS PARA GENTE MUY POBRE




---



#### Types of Bathroom Facilities Across Poverty Levels

In [ ]:
bathroom_types = ['sanitario1', 'sanitario2', 'sanitario3', 'sanitario5', 'sanitario6']
bathroom_names = ['No Toilet', 'Sewer/Cesspool', 'Septic Tank', 'Black Hole/Latrine', 'Other System']

fig = make_subplots(rows=2, cols=3, subplot_titles=bathroom_names)
row, col = 1, 1

for bathroom_type, name in zip(bathroom_types, bathroom_names):
    df_train[f'{bathroom_type}_label'] = df_train[bathroom_type].apply(replace_binary)
    df_bathroom = df_train.groupby(['Poverty Level', f'{bathroom_type}_label']).size().unstack()
    df_bathroom_perc = df_bathroom.div(df_bathroom.sum(axis=1), axis=0) * 100

    for i in df_bathroom_perc.columns:
        fig.add_trace(
            go.Bar(
                x=df_bathroom_perc.index,
                y=df_bathroom_perc[i],
                name=f'{name}_{i}',
                text=df_bathroom_perc[i].round(1),
                textposition='auto',
                hoverinfo='text',
                hovertext=[f'{val:.1f}%' for val in df_bathroom_perc[i]]
            ),
            row=row, col=col
        )

    fig.update_xaxes(title_text='Poverty Level', row=row, col=col)
    fig.update_yaxes(title_text='Percentage', row=row, col=col)

    col += 1
    if col > 3:
        col = 1
        row += 1

fig.update_layout(height=800, width=1200, title_text="Types of Bathroom Facilities Across Poverty Levels")
fig.show()



---



#### (Fran) Age Distribution Across Poverty Levels

In [ ]:
fig = go.Figure()

for poverty_level in df_train['Poverty Level'].unique():
    data = df_train[df_train['Poverty Level'] == poverty_level]['age']
    fig.add_trace(go.Violin(y=data, x=[poverty_level]*len(data), name=poverty_level, box_visible=True, meanline_visible=True))

fig.update_layout(
    title='Age Distribution Across Poverty Levels',
    yaxis_title='Age',
    xaxis_title='Poverty Level',
    height=600,
    width=800
)

fig.show()

#### (Fran) Asset Ownership Across Poverty Levels

In [ ]:
assets = ['television', 'mobilephone', 'owns_tablet']  # v18q is for tablet ownership
asset_names = ['Television', 'Mobile Phone', 'Tablet']

fig = make_subplots(rows=1, cols=3, subplot_titles=asset_names)

for i, (asset, name) in enumerate(zip(assets, asset_names), 1):
    df_asset = df_train.groupby('Poverty Level')[asset].mean() * 100

    fig.add_trace(
        go.Bar(x=df_asset.index, y=df_asset.values, name=name, text=df_asset.values.round(1),
               textposition='auto', hoverinfo='text',
               hovertext=[f'{val:.1f}%' for val in df_asset.values]),
        row=1, col=i
    )

    fig.update_xaxes(title_text='Poverty Level', row=1, col=i)
    fig.update_yaxes(title_text='Percentage', row=1, col=i)

fig.update_layout(height=500, width=1200, title_text="Asset Ownership Across Poverty Levels")
fig.show()



---



#### (Fran) Poverty Level Distribution by Region

In [ ]:
# Geographic Distribution of Poverty
regions = ['region_central', 'region_chorotega', 'region_pacifico_central', 'region_brunca', 'region_huetar_atlantica', 'region_huetar_norte']
region_names = ['Central', 'Chorotega', 'Pacifico Central', 'Brunca', 'Huetar Atlantica', 'Huetar Norte']

fig = go.Figure()

for region, name in zip(regions, region_names):
    poverty_distribution = df_train[df_train[region] == 1]['Poverty Level'].value_counts().sort_index()
    poverty_distribution_percentage = poverty_distribution / poverty_distribution.sum() * 100

    fig.add_trace(go.Bar(
        x=poverty_distribution_percentage.index,
        y=poverty_distribution_percentage.values,
        name=name,
        text=[f'{val:.1f}%' for val in poverty_distribution_percentage.values],
        textposition='auto'
    ))

fig.update_layout(
    title='Poverty Level Distribution by Region',
    xaxis_title='Poverty Level',
    yaxis_title='Percentage',
    barmode='group',
    height=600,
    width=1000
)

fig.show()



---



#### Urban vs Rural Poverty Rates

In [ ]:
# Urban vs Rural Poverty Rates
urban_poverty = df_train[df_train['urban_zone'] == 1]['Poverty Level'].value_counts().sort_index()
rural_poverty = df_train[df_train['rural_zone'] == 1]['Poverty Level'].value_counts().sort_index()

urban_poverty_percentage = urban_poverty / urban_poverty.sum() * 100
rural_poverty_percentage = rural_poverty / rural_poverty.sum() * 100

fig = go.Figure()

fig.add_trace(go.Bar(
    x=urban_poverty_percentage.index,
    y=urban_poverty_percentage.values,
    name='Urban',
    text=[f'{val:.1f}%' for val in urban_poverty_percentage.values],
    textposition='auto'
))

fig.add_trace(go.Bar(
    x=rural_poverty_percentage.index,
    y=rural_poverty_percentage.values,
    name='Rural',
    text=[f'{val:.1f}%' for val in rural_poverty_percentage.values],
    textposition='auto'
))

fig.update_layout(
    title='Urban vs Rural Poverty Rates',
    xaxis_title='Poverty Level',
    yaxis_title='Percentage',
    barmode='group',
    height=600,
    width=800
)

fig.show()



---



#### Types of Water Access Across Poverty Levels

In [ ]:
water_types = ['abastaguadentro', 'abastaguafuera', 'abastaguano']
water_names = ['Inside House', 'Outside House', 'No Water Access']

fig = go.Figure()

for water_type, name in zip(water_types, water_names):
    data = df_train.groupby('Poverty Level')[water_type].mean() * 100
    fig.add_trace(go.Bar(x=data.index, y=data.values, name=name,
                         text=[f'{val:.1f}%' for val in data.values],
                         textposition='auto'))

fig.update_layout(
    title='Types of Water Access Across Poverty Levels',
    xaxis_title='Poverty Level',
    yaxis_title='Percentage',
    barmode='group',
    height=600,
    width=900
)

fig.show()



---



#### Types of Housing by Poverty Levels

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Define housing types and their names
housing_types = ['own_fully_paid', 'own_paying_installments', 'rented', 'precarious', 'other_housing']
housing_names = ['Fully Paid', 'Paying Installments', 'Rented', 'Precarious', 'Other']

# Create subplots, one for each poverty level
fig = make_subplots(rows=2, cols=2, subplot_titles=['Extreme Poverty', 'Moderate Poverty', 'Vulnerable', 'Non-Vulnerable'])

# Define positions for subplots
positions = [(1, 1), (1, 2), (2, 1), (2, 2)]

# Iterate through poverty levels
for i, poverty_level in enumerate(['Extreme Poverty', 'Moderate Poverty', 'Vulnerable', 'Non-Vulnerable'], 1):
    # Filter data for the current poverty level
    poverty_data = df_train[df_train['Poverty Level'] == poverty_level]

    # Calculate the percentage of each housing type
    housing_distribution = poverty_data[housing_types].mean() * 100

    # Add a bar trace for the current poverty level
    fig.add_trace(
        go.Bar(
            x=housing_names,
            y=housing_distribution,
            name=poverty_level,
            text=[f'{val:.1f}%' for val in housing_distribution],
            textposition='auto'
        ),
        row=positions[i-1][0], col=positions[i-1][1]
    )

# Update layout
fig.update_layout(
    title='Types of Housing by Poverty Level',
    height=800,
    width=1200,
    showlegend=False
)

# Update axes
for i in range(1, 5):
    fig.update_xaxes(title_text='Housing Type', row=positions[i-1][0], col=positions[i-1][1])
    fig.update_yaxes(title_text='Percentage', row=positions[i-1][0], col=positions[i-1][1])

fig.show()

En general, las viviendas propias y completamente pagadas tienen la mayor proporción de "No vulnerable", que es el nivel de pobreza más bajo. Esto implica que tener una vivienda propia y sin deudas está relacionado con un nivel de pobreza más bajo.

Por otro lado, la vivienda con la mayor cantidad de "Pobreza Extrema" (nivel más alto de pobreza) es "precaria", lo que significa viviendas precarias o con condiciones deficientes. Esto sugiere que las personas que viven en este tipo de hogares tienen más probabilidades de estar en situación de pobreza extrema.

Las viviendas "own_paying_installments", es decir, las viviendas propias pero que aún se están pagando en cuotas, tienen una distribución más equitativa entre los diferentes niveles de pobreza, con una ligera tendencia hacia "No vulnerable".

En comparación con las viviendas propias, las viviendas arrendadas tienen una mayor proporción de hogares en los niveles de "vulnerabilidad" y "pobreza moderada".

El tipo de hogar "other_housing", que podría incluir viviendas prestadas, cedidas o en otras circunstancias, muestra una distribución más uniforme entre los diferentes niveles de pobreza, con una ligera inclinación hacia "vulnerable" y "pobreza moderada".


---



#### (Fran) Overcrowding Distribution Across Poverty Levels

Aqui se va a medir como independientemente, la cantidad de personas por hogar y si

In [ ]:
fig = go.Figure()

for poverty_level in df_train['Poverty Level'].unique():
    data = df_train[df_train['Poverty Level'] == poverty_level]['overcrowding']
    fig.add_trace(go.Box(y=data, name=poverty_level, boxpoints='all', jitter=0.3, pointpos=-1.8))

fig.update_layout(
    title='Overcrowding Distribution Across Poverty Levels',
    yaxis_title='Overcrowding',
    xaxis_title='Poverty Level',
    height=600,
    width=800
)

fig.show()



---



# AQUI AGREGAR EN BASE A LOS GRÁFICOS ALGUNAS CONCLUSIONES, QUE VARIABLES SE VE QUE SON INÚTILES (TODAS SON FALSE PARA TODOS) Y CUALES TIENEN POTENCIAL


---

### Relaciones entre Variables (Without Target)

---

---

#### (REPETIDO) Relación entre años de educación y Nivel pobreza

Ahora, se pretende analizar la relación que existe entre los años de educación y el nivel de pobreza según el *target*. Para esto se va a utilizar una representación llamada *boxplot* que permite visualizar la distribución de los años de escolaridad para cada nivel de pobreza de acuerdo con el *Target*.

En un *boxplot*, las cajas rectangulares representan la distribución de los años de escolaridad para un valor de *Target*, donde la linea dentro de la caja representa la mediana. También se pueden observar los limites inferior y superior de la caja. El 50% de los datos se encuentran dentro de la caja. Las lineas que se extienden de la caja, son conocidad como *bigotes* y estas muestran la dispersión de los datos, y en cuanto a los puntos, estos son los valores atípicos que se encuentran lejos de el resto de datos (no siempre están presentes).

En el gráfico, se puede observar perfectamente la relación que existe entre estas 2 variables y el impacto que tienen los años de escolaridad con el nivel de pobreza en este dataset.

Lo que este *boxplot* muestra, es que entre más años de escolaridad, menor va a ser el nivel de pobreza. Y es que se puede ver que conforme aumenta el valor del *Target*, tambíen aumentan los años promedio de educación. Por lo tanto, como era de esperar, los años de escolaridad sí tienen un impacto significativo sobre el nivel de pobreza.

Para agregar más puntos que pueden ser interesantes como parte del análisis de esta relación, la dispersión de los datos también aumenta conforme el *Target* aumenta, y los valores atípicos que existen se encuentran particularmente en los *Target 2, Target 3 y Target 4*, donde algunos datos están por encima del resto.

In [ ]:
# Gráfico 2: Años de escolaridad promedio por grupo de Target
meaneduc_mean = df_train.groupby('Target')['meaneduc'].mean().values
names = ["Extereme Poverty","Moderate Poverty", "Vulnerable", "NonVulnerable"]
trace = go.Bar(x=names, y=meaneduc_mean, marker=dict(color='orange', opacity = 0.6))
layout = go.Layout(title='Escolaridad Promedio por Nivel de Pobreza',
                   xaxis=dict(title='Target'),
                   yaxis=dict(title='Años de Escolaridad Promedio'), margin=dict(l=100), width=1000, height=600)
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

Utilizando la variable "meaneduc" (promedio de años de educación de un adulto), se puede observar claramente que niveles más altos de educación distinguen cada nivel de pobreza del otro. Un promedio de 7 años de educación marca la diferencia entre la clase 1 y la clase 2. Un promedio de 8 años separa a la clase 2 de la clase 3. Y promedios superiores a 8 años de educación indican que es muy probable que se pertenezca a la clase 4, la de menor pobreza.

Claramente los años promedio de educación actúan como un indicador confiable para diferenciar los distintos estratos socioeconómicos. A mayor nivel educativo alcanzado, menor es el grado de pobreza experimentado

---

---

#### Average Education Levels by Region

Ya que se sabe que el nivel de educación afecta los niveles de pobreza, se va a hacer un análisis de los diferentes niveles de educación por región, para ver si alguna región en particular será más útil para detectar los niveles de pobreza

A continuación se va a realizar un análisis sobre la relación que pueda existir entre los *niveles de educación promedio* y la *región*. La columna del dataset **df_train** que representa los *niveles de educación promedio* es la de 'meaneduc'. Esta columna hace referencia a los años de educación promedio de los adultos en una vivienda.

En cuanto a la región, hay varias columnas que representan las regiones de Costa Rica:

* 'region_central'
* 'region_chorotega'
* 'region_pacifico_central'
* 'region_brunca'
* 'region_huetar_atlantica'
* 'region_huetar_norte'


Por medio de un *barplot* se pretende poder visualizar y determinar qué relación existen entres estas variables.

In [ ]:
regions = ['region_central', 'region_chorotega', 'region_pacifico_central', 'region_brunca', 'region_huetar_atlantica', 'region_huetar_norte']
region_names = ['Central', 'Chorotega', 'Pacifico Central', 'Brunca', 'Huetar Atlantica', 'Huetar Norte']

fig = go.Figure()

for region, name in zip(regions, region_names):
    avg_education = df_train[df_train[region] == 1]['meaneduc'].mean()
    fig.add_trace(go.Bar(x=[name], y=[avg_education], name=name,
                         text=[f'{avg_education:.2f}'],
                         textposition='auto'))

fig.update_layout(
    title='Average Education Levels by Region',
    xaxis_title='Region',
    yaxis_title='Average Years of Education',
    height=600,
    width=900
)

fig.show()

En el gráfico anterior se puede observar que las regiones con un mayor promedio de años de educación son las de  **Región Huetar Norte** y **Región Huetar Atlantica**. Por otro lado, las regiones con un menor promedio de años de educación son las regiones de **Región Central** y **Región Chorotega**.

De acuerdo a lo anterior, se puede interpretar que existe una conexión entre niveles de pobreza y la región, en especial si se tiene en cuenta el impacto de educación sobre niveles de pobreza

---
---

#### Types of Housing by Region

In [ ]:
housing_types = ['own_fully_paid', 'own_paying_installments', 'rented', 'precarious', 'other_housing']
housing_names = ['Fully Paid', 'Paying Installments', 'Rented', 'Precarious', 'Other']

fig = go.Figure()

for region, region_name in zip(regions, region_names):
    region_data = df_train[df_train[region] == 1]
    housing_distribution = region_data[housing_types].mean() * 100
    fig.add_trace(go.Bar(x=housing_names, y=housing_distribution, name=region_name,
                         text=[f'{val:.1f}%' for val in housing_distribution],
                         textposition='auto'))

fig.update_layout(
    title='Types of Housing by Region',
    xaxis_title='Housing Type',
    yaxis_title='Percentage',
    barmode='group',
    height=600,
    width=1000
)

fig.show()

El gráfico muestra la distribución proporcional de los diferentes tipos de vivienda en cada una de las regiones de Costa Rica. A partir de esta visualización, se pueden extraer algunas conclusiones sobre la relación entre el tipo de vivienda predominante en cada región y los niveles de pobreza asociados.

* En las regiones central y pacifico central, se observa una mayor proporción de viviendas propias totalmente pagadas (own_fully_paid) y en proceso de pago (own_paying_installments). Esto sugiere que en estas regiones, una gran parte de la población tiene acceso a vivienda propia, lo que podría estar asociado con menores niveles de pobreza.

* En la region Brunca, se aprecia una distribución más equilibrada entre los diferentes tipos de vivienda, con una presencia significativa de viviendas alquiladas (rented) y en menor medida, viviendas precarias (precarious).
Comparada con el resto, esta región tiene una proporción relativamente alta de viviendas precarias (precarious). Dado que estos tipo de viviendas se asociam con mayores niveles de pobreza, esto sugiere que la región Brunca podría enfrentar desafíos socioeconómicos más pronunciados.

* De todas las regiones que se pueden observar, la región con menos hogares en condición precaria es la huetar atlántica, esto también es indicador de que aqui probablemente los niveles de pobreza son menores




---



#### (Fran) Relationship between People per Household and Overcrowding by bedrooms

To carry out the analysis of the relationship between **People per Household and Overcrowding by bedrooms**, we will use a boxplot to easily visualize the distribution and behavior of the data. What we are looking for is if the total number of people in a household has any impact or relationship with the existance of overcrowded bedrooms.

In [ ]:
overcrowding_bedrooms = df_train['overcrowding_bedrooms']
total_persons = df_train['total_persons']

trace = go.Box(x=overcrowding_bedrooms, y=total_persons, marker=dict(color='green', opacity = 0.6), jitter=0.2)
layout = go.Layout(title='Relationship between People per Household and Overcrowding',
                   xaxis=dict(title='Overcrowding'),
                   yaxis=dict(title='People per Household'), margin=dict(l=100), width=1000, height=600)
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

After analyzing the previous graph, we notice that when the total number of people in a household increases there is more overcrowded bedrooms. Now, remember the following:

* 0 = NO overcrowded bedrooms
* 1 = overcrowded bedrooms

This observation makes sense because, according to the graph, when there is overcrowding, the total number of people is typically higher than 4 and the median distribution is 6. This suggests that when the number of people in a household is high enough, there are more posibilities that the household have overcrowded bedrooms. Therefore, there is a strong relationship between these two variables.

---

---

#### (Fran) Relationship between Overcrowding and Economic Dependency

A continuación se realiza un *scatterplot* con el fin de ver la relación que existe entre el *overcrowding* (sobrepoblación) y la dependencia económica.

In [ ]:
hexbin_trace = go.Histogram2d(
    x=df_train['overcrowding'],
    y=df_train['dependency'],
    colorscale='YlOrRd',
    nbinsx=30,
    nbinsy=30,
    colorbar=dict(title='Point'),
)

# Crear el layout
layout = go.Layout(
    title='Relationship between Overcrowding and Economic Dependency',
    xaxis=dict(title='Overcrowding'),
    yaxis=dict(title='Economic Dependency'),
    width=1000,
    height=600,
    margin=dict(l=50, r=50, t=50, b=50),
    paper_bgcolor='white',
    plot_bgcolor='white'
)

# Crear la figura
fig = go.Figure(data=[hexbin_trace], layout=layout)

# Mostrar el gráfico
fig.show()

A partir de lo anterior, en el gráfico no se puede observar ningún patrón o tendencia que permita mostrar una asociación entre ambas variables. De hecho se puede visualizar como los datos están dispersos y no hay alguna tendencia lineal. Sin embargo, esto no quiere decir que no exista una relación entre ambas variables.

Por otro lado, este gráfico permite demostrar la necesidad de reemplazar los "yes" y "no" en la columna de 'dependency' por valores númericos en la sección de **preparación de los datos**, ya que estos tipos de valores pueden afectar el entrenamiento de los modelos.

### 1.3.1 Informe de Exploración de Datos
*¿Qué tipo de hipótesis sobre los datos ha formulado?*

Después de una exploración inicial de los datos, se pueden formular las siguientes hipótesis:

Variables como meaneduc (educación promedio de adultos), overcrowding (hacinamiento) y dependency (tasa de dependencia) probablemente tengan una fuerte correlación con el nivel de pobreza (target). Un mayor nivel educativo, menor hacinamiento y menor dependencia deberían asociarse con menores niveles de pobreza.
La tenencia y tipo de vivienda (tipovivi1 a tipovivi5) también podrían ser predictores importantes. La posesión de una vivienda propia y totalmente pagada (tipovivi1) debería correlacionar con menor pobreza en comparación con viviendas rentadas, precarias u otras.
Características de la vivienda como el tipo de paredes, piso y techo podrían reflejar condiciones de precariedad asociadas con mayor pobreza. Variables como paredblolad, pisomoscer, etc. codificadas como 0 (ausencia) podrían indicar carencias.
El acceso a servicios públicos y sanitarios (abastaguadentro, public, sanitario1, elimbasu1, etc.) también podría ser un diferenciador clave. La falta de estos servicios básicos probablemente se asocie con niveles más altos de pobreza.
La región (lugar1 a lugar6) y la zona (urbana/rural) podrían mostrar disparidades en los niveles de pobreza, con ciertas regiones y áreas rurales teniendo mayor incidencia.

*¿Qué atributos parecen ser prometedores de cara a futuros análisis?*

En base a las hipótesis formuladas y la exploración inicial, algunos de los atributos más prometedores para predecir el nivel de pobreza parecen ser:

meaneduc: la educación promedio de los adultos en el hogar.
overcrowding: el nivel de hacinamiento medido por personas por habitación.
dependency: la tasa de dependencia económica en el hogar.
tipovivi1 a tipovivi5: el tipo y tenencia de la vivienda.
Variables de calidad de la vivienda como paredblolad, pisomoscer, etc.
Variables de acceso a servicios como abastaguadentro, public, sanitario1, elimbasu1.
región (lugar1 a lugar6) y zona (urbana/rural).

*¿Ha realizado exploraciones que revelen nuevas características de los datos?*

Al analizar la distribución de la variable objetivo (Target), se observó que las clases no están balanceadas. Hay una predominancia de hogares no vulnerables (target = 4), mientras que la clase de pobreza extrema (target = 1) es minoritaria. Esto podría requerir técnicas de re-muestreo para evitar un sesgo del modelo hacia la clase mayoritaria.
También se identificaron varias columnas con una proporción significativa de valores faltantes, como v2a1 (renta mensual), v18q1 (cantidad de tablets), rez_esc (rezago escolar) y meaneduc (educación promedio). Será crucial definir una estrategia para manejar estos valores ausentes.
Además, se encontraron algunas inconsistencias en los esquemas de codificación, como en las variables edjefe/edjefa y dependency que mezclan valores numéricos con 'yes' y 'no'. Esto requerirá un preprocesamiento especial.


¿En qué forma han cambiado estas exploraciones su hipótesis inicial?

Las exploraciones han reforzado varias de las hipótesis iniciales, particularmente en cuanto a la importancia de variables como la educación, el hacinamiento, la dependencia económica y las características de la vivienda para predecir la pobreza.
Sin embargo, también han revelado nuevos desafíos y consideraciones, como la necesidad de manejar el desbalance de clases en la variable objetivo, la presencia de valores faltantes significativos en varias columnas, y algunas inconsistencias en los esquemas de codificación que requerirán atención en la fase de preprocesamiento.

¿Puede identificar subconjuntos concretos de datos para un uso posterior?
Basado en las exploraciones, se podrían considerar los siguientes subconjuntos de datos para análisis posteriores:

Un subconjunto con las variables más prometedoras identificadas, como meaneduc, overcrowding, dependency, tipovivi, etc., para enfocarse en los predictores clave.
Subconjuntos basados en la región o la zona (urbana/rural) para analizar patrones y disparidades específicas de cada contexto.
Subconjuntos con los registros completos (sin valores faltantes) en las variables clave, para evitar la necesidad de imputación en una primera instancia.
Submuestras balanceadas de la variable objetivo (Target) para entrenar modelos que no estén sesgados hacia la clase mayoritaria.

*Vuelva a comprobar sus objetivos de minería de datos. ¿Esta exploración ha modificado sus objetivos?*

La exploración ha proporcionado una comprensión más profunda de los datos y ha revelado tanto oportunidades como desafíos para el modelado predictivo de la pobreza.
Si bien el objetivo general sigue siendo desarrollar un modelo que pueda predecir con precisión el nivel de pobreza de los hogares basado en sus características socioeconómicas y de vivienda, la exploración ha resaltado la necesidad de objetivos adicionales en el preprocesamiento de los datos, como la gestión de valores faltantes, la codificación consistente de variables categóricas y el manejo del desbalance de clases.
Además, la exploración sugiere que podría ser valioso no solo desarrollar un modelo global, sino también analizar patrones y factores de pobreza específicos para diferentes regiones o zonas, lo que podría constituir un objetivo adicional de minería de datos.
En resumen, si bien los objetivos centrales se mantienen, la exploración ha proporcionado una hoja de ruta más clara de los pasos necesarios para alcanzarlos y ha sugerido algunas direcciones adicionales que podrían enriquecer el análisis.

---

## 1.4 Aseguramiento calidad de los Datos



En esta sección, se realizará un análisis detallado de las columnas del dataframe para obtener información clave sobre la calidad y composición de los datos. Se examinarán los siguientes aspectos:

* Tipos de datos: Se identificará el tipo de datos (integer, float, string, object) de cada columna.

* Recuento de tipos de datos: Se determinará la cantidad de columnas que contienen cada tipo de dato.

* Datos faltantes: Se detectarán las columnas con valores faltantes y se calculará la cantidad de valores nulos en cada una.

### 1.4.1 Tipos de datos Y Recuento

Aquí se llama una función que lista el tipo de dato de cada columna dentro del frame y además se muestran la cantidad de columnas para cada tipo:



In [ ]:
# Tipos de cada columna
df_train.dtypes

Id                        object
monthly_rent             float64
overcrowding_bedrooms      int64
total_rooms                int64
overcrowding_rooms         int64
                          ...   
sanitario1_label          object
sanitario2_label          object
sanitario3_label          object
sanitario5_label          object
sanitario6_label          object
Length: 173, dtype: object

Sin embargo, esto aún no refleja nada, por lo que lo mejor es realizar un conteo de columnas por cada tipo de datos. De esta forma, se podrá visualizar e identificar qué tipos de datos componen el dataset.

In [ ]:
# Obtener los tipos de datos de cada columna
column_types = df_train.dtypes

# Contar las columnas por cada tipo de dato
type_counts = column_types.value_counts()

print("Conteo de columnas por tipo de dato:")
print(type_counts)

Conteo de columnas por tipo de dato:
int64      130
object      35
float64      8
Name: count, dtype: int64


Se puede observar que hay 3 tipos de datos (int64, float64 y object), se van a explorar las columnas con cada uno de los tipos de datos para ver si realmente estos son los tipos adecuados para lo que se espera obtener de cada columna:

Primero se realiza una exploración sobre aquellas columnas que son de tipo *int64*. Se puede observar que sí es correcto que todas estas columnas sean consideradas de este tipo, ya que son variables númericas que necesitan ser representandas como un número entero.

In [ ]:
df_train.loc[:, df_train.dtypes == "int64"]

,overcrowding_bedrooms,total_rooms,overcrowding_rooms,bathroom,refrigerator,owns_tablet,males_under_12,males_over_12,total_males,females_under_12,...,urban_zone,rural_zone,age,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,agesq,Target
0,0,3,0,1,1,0,0,1,1,0,...,1,0,43,100,1849,1,100,0,1849,4
1,0,4,0,1,1,1,0,1,1,0,...,1,0,67,144,4489,1,144,0,4489,4
2,0,8,0,1,1,0,0,0,0,0,...,1,0,92,121,8464,1,0,0,8464,4
3,0,5,0,1,1,1,0,2,2,1,...,1,0,17,81,289,16,121,4,289,4
4,0,5,0,1,1,1,0,2,2,1,...,1,0,37,121,1369,16,121,4,1369,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9552,0,6,0,1,1,0,0,2,2,1,...,0,1,46,81,2116,25,81,1,2116,2
9553,0,6,0,1,1,0,0,2,2,1,...,0,1,2,0,4,25,81,1,4,2
9554,0,6,0,1,1,0,0,2,2,1,...,0,1,50,25,2500,25,81,1,2500,2
9555,0,6,0,1,1,0,0,2,2,1,...,0,1,26,121,676,25,81,1,676,2


In [ ]:
object_columns_train = df_train.columns[df_train.dtypes == 'int64']
object_columns_train

Index(['overcrowding_bedrooms', 'total_rooms', 'overcrowding_rooms',
       'bathroom', 'refrigerator', 'owns_tablet', 'males_under_12',
       'males_over_12', 'total_males', 'females_under_12',
       ...
       'urban_zone', 'rural_zone', 'age', 'SQBescolari', 'SQBage',
       'SQBhogar_total', 'SQBedjefe', 'SQBhogar_nin', 'agesq', 'Target'],
      dtype='object', length=130)

---

Ahora se realiza los mismo con aquellas columnas de tipo *float64* y al igual que en las anteriores, la mayoría tienen sentido que sean de este tipo ya que son variables númericas como *monthly_rent* o *meaneduc* que necesitan representarse en decimales para capturar su presición adecuadamente, lo que beneficia a las distribuciones y variabilidad de los datos.

Sin embargo, también se puede observar que la variable de *num_tablets no deberia de ser de tipo *float* porque lo que representa es el número de tablets que hay en una casa, lo que debería de representarse de manera exacta y precisa con un número entero.


In [ ]:
df_train.loc[:, df_train.dtypes == "float64"]

,monthly_rent,num_tablets,rez_esc,meaneduc,overcrowding,SQBovercrowding,SQBdependency,SQBmeaned
0,190000.0,NaN,NaN,10.00,1.000000,1.000000,0.0000,100.0000
1,135000.0,1.0,NaN,12.00,1.000000,1.000000,64.0000,144.0000
2,NaN,NaN,NaN,11.00,0.500000,0.250000,64.0000,121.0000
3,180000.0,1.0,1.0,11.00,1.333333,1.777778,1.0000,121.0000
4,180000.0,1.0,NaN,11.00,1.333333,1.777778,1.0000,121.0000
...,...,...,...,...,...,...,...,...
9552,80000.0,NaN,NaN,8.25,1.250000,1.562500,0.0625,68.0625
9553,80000.0,NaN,NaN,8.25,1.250000,1.562500,0.0625,68.0625
9554,80000.0,NaN,NaN,8.25,1.250000,1.562500,0.0625,68.0625
9555,80000.0,NaN,NaN,8.25,1.250000,1.562500,0.0625,68.0625


In [ ]:
object_columns_train = df_train.columns[df_train.dtypes == 'float64']
object_columns_train

Index(['monthly_rent', 'num_tablets', 'rez_esc', 'meaneduc', 'overcrowding',
       'SQBovercrowding', 'SQBdependency', 'SQBmeaned'],
      dtype='object')

---

Por último, se realiza la misma exploración con el tipo de dato *object*, en este es muy interesante ver que lo que se tiene son dos columnas de *identificadores* (IDHogar y ID) una propia del dataset y otra para los hogares.

Estas 2 columnas se limpiaran y muy problemente no seleccionaran en la sección de preparación de datos, ya que al tratarse de identificadores, estos tienen un aporte nulo en relación con las demás variables y por lo tanto los modelos **no** deberían de ser entrenados con estas variables.

También se puede observar 3 columnas que fueron asignadas como tipo object, ya que cuentan con registros tanto numericos como con strings ('yes' y 'no', estos sin ninguna duda buscan representar de manera binaria un 1 y 0). Dado esto, lo apropiado va a ser que en la **sección de preparación de los datos** se cambien los 'yes' y 'no' por unos y ceros, y posteriormente convertir las variables a tipo *int64* en el caso de *edjefe y edjefa*, y a tipo *float64* en el caso de *dependency* porque esta se trata de un rango de dependencia en el que sus calculos surgen de una división que retorna número flotantes.

In [ ]:
df_train.loc[:, df_train.dtypes == "object"]

,Id,idhogar,dependency,edjefe,edjefa,Poverty Level,paredblolad_label,paredzocalo_label,paredpreb_label,pareddes_label,...,elimbasu2_label,elimbasu3_label,elimbasu4_label,elimbasu5_label,elimbasu6_label,sanitario1_label,sanitario2_label,sanitario3_label,sanitario5_label,sanitario6_label
0,ID_279628684,21eb7fcc1,no,10,no,Non-Vulnerable,True,False,False,False,...,False,False,False,False,False,False,True,False,False,False
1,ID_f29eb3ddd,0e5d7a658,8,12,no,Non-Vulnerable,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
2,ID_68de51c94,2c7317ea8,8,no,11,Non-Vulnerable,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
3,ID_d671db89c,2b58d945f,yes,11,no,Non-Vulnerable,True,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,ID_d56d6f5f5,2b58d945f,yes,11,no,Non-Vulnerable,True,False,False,False,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9552,ID_d45ae367d,d6c086aa3,.25,9,no,Moderate Poverty,True,False,False,False,...,True,False,False,False,False,False,False,True,False,False
9553,ID_c94744e07,d6c086aa3,.25,9,no,Moderate Poverty,True,False,False,False,...,True,False,False,False,False,False,False,True,False,False
9554,ID_85fc658f8,d6c086aa3,.25,9,no,Moderate Poverty,True,False,False,False,...,True,False,False,False,False,False,False,True,False,False
9555,ID_ced540c61,d6c086aa3,.25,9,no,Moderate Poverty,True,False,False,False,...,True,False,False,False,False,False,False,True,False,False


In [ ]:
object_columns_train = df_train.columns[df_train.dtypes == 'object']
object_columns_train

Index(['Id', 'idhogar', 'dependency', 'edjefe', 'edjefa', 'Poverty Level',
       'paredblolad_label', 'paredzocalo_label', 'paredpreb_label',
       'pareddes_label', 'pisomoscer_label', 'pisocemento_label',
       'pisoother_label', 'pisonatur_label', 'pisonotiene_label',
       'pisomadera_label', 'techozinc_label', 'techoentrepiso_label',
       'techocane_label', 'techootro_label', 'public_label', 'planpri_label',
       'noelec_label', 'coopele_label', 'elimbasu1_label', 'elimbasu2_label',
       'elimbasu3_label', 'elimbasu4_label', 'elimbasu5_label',
       'elimbasu6_label', 'sanitario1_label', 'sanitario2_label',
       'sanitario3_label', 'sanitario5_label', 'sanitario6_label'],
      dtype='object')



---



---



### 1.4.2 Datos Faltantes

#### 1.4.2.1 Columnas con valores nulos

Se realiza el siguiente procedimiento para identificar cuáles columnas contienen valores nulos y cuántos en valores nulos en total.

In [ ]:
#Mostrar columnas con valores nulos en el dataset de entrenamiento
columns_with_nulls_train = df_train.columns[df_train.isnull().any()]
print("Columnas con valores nulos en el dataset de entrenamiento:")
columns_with_nulls_train

print("Número de valores nulos en las columnas con valores nulos (dataset de entrenamiento):")

for column in columns_with_nulls_train:
    print(f"{column}: {df_train[column].isnull().sum()}")

Columnas con valores nulos en el dataset de entrenamiento:
Número de valores nulos en las columnas con valores nulos (dataset de entrenamiento):
monthly_rent: 6860
num_tablets: 7342
rez_esc: 7928
meaneduc: 5
SQBmeaned: 5


 A partir de esto, se puede observar que en el dataset hay 5 columnas con valores nulos, donde

 * monthly_rent
 * num_tablets
 * rez_esc

tienen una cantidad considerable de valores nulos, los cuales se deben abordar en la sección de preparación de datos para conseguir reducir o eliminar esa cantidad de valores.

También se puede observar que hay dos columnas *meaneduc y SQBmeaned* que solo tienen 5 filas en las que cuentan con valores nulos, por lo que en la preparación de datos se puede abordar eliminando esas 5 filas, dado que la eliminación de 5 o 10 filas de un conjunto de datos que contiene más de 9000 registros no tendrá un impacto considerable en los resultados de los datos.

#### 1.4.2.2 Filas del Dataset duplicados


In [ ]:
# Verificar si hay filas con ID duplicados
duplicated_ids = df_train[df_train['Id'].duplicated()]

if len(duplicated_ids) > 0:
    print(f"Se encontraron {len(duplicated_ids)} filas con ID duplicados:")
    print(duplicated_ids)
else:
    print("No se encontraron filas con ID duplicados.")

No se encontraron filas con ID duplicados.


Al realizar un búsqueda sobre el dataset **df_train** se pudó comprobar que no existen identificadores duplicados en ninguna de las filas de todo el conjunto de datos.



---



---
### 1.4.3 Informe de Aseguramiento de la Calidad

**Tipos de datos y Recuento:**

Al analizar los tipos de datos de cada columna, se encontró lo siguiente:

* 130 columnas son de tipo int64, representando variables numéricas enteras.
* 8 columnas son de tipo float64, para variables numéricas con decimales.
* 5 columnas son de tipo object, generalmente usado para variables categóricas o de texto.

Se realizó una exploración más detallada de las columnas por tipo de dato:

* **int64:** la mayoría de estas columnas efectivamente representan cantidades o variables categóricas codificadas numéricamente, como número de habitaciones, número de personas, servicios disponibles (1 o 0), etc. Los tipos de datos son apropiados.

* **float64:** aquí se encuentran variables como la renta mensual, tasa de dependencia, educación promedio, etc. que tienen sentido como números con decimales. Sin embargo, la columna 'num_tablets' que representa la cantidad de tablets, debería ser un entero y no un flotante.

* **object:** en este tipo se encuentran los identificadores 'Id' e 'idhogar' que son cadenas alfanuméricas. Además, hay 3 columnas 'dependency', 'edjefe', 'edjefa' que contienen una mezcla de valores numéricos y las cadenas 'yes', 'no', lo cual requerirá un tratamiento especial para homogeneizar los tipos de datos.


**Datos Faltantes:**

Después de analizar las columnas en busca de valores faltantes, se encontró que hay 5 columnas con valores nulos:

* '**v2a1**' (renta mensual): 6860 valores nulos
* '**v18q1**' (número de tablets): 7342 valores nulos
* '**rez_esc**' (años de retraso escolar): 7928 valores nulos
* '**meaneduc**' (educación promedio de adultos): 5 valores nulos
* '**SQBmeaned**' (educación promedio al cuadrado): 5 valores nulos

Las primeras 3 columnas tienen una cantidad muy significativa de valores faltantes, representando más del 70% de los registros en cada caso. Esto requerirá una estrategia para manejar los valores ausentes, ya sea imputando los valores o considerando la eliminación de estas columnas si la pérdida de información no es crítica.

Por otro lado, 'meaneduc' y 'SQBmeaned' solo tienen 5 valores nulos cada una, una proporción muy pequeña del total de registros. En este caso, los registros con estos valores faltantes podrían simplemente eliminarse sin un impacto significativo en el análisis.

Además, se verificó que no haya filas duplicadas en base al identificador 'Id', y efectivamente no se encontraron duplicados.


---